In [15]:
from langchain_community.document_loaders import UnstructuredURLLoader
urls = ['https://www.plutodaycare.com/',
        'https://www.plutodaycare.com/about-us',
        'https://www.plutodaycare.com/locations',
        ]
loader = UnstructuredURLLoader(urls=urls)
data = loader.load()  

In [16]:
data

[Document(metadata={'source': 'https://www.plutodaycare.com/'}, page_content='Welcome to Pluto Day Care\n\nYour Day Care\n\nAbout\n\nHow We Work\n\nAt Pluto Day Care, we are dedicated to creating a nurturing, stimulating and welcoming environment for all children. Our unique programs incorporate a variety of educational theories, while always keeping the child’s best interest in mind and allowing for hands-on, experiential learning. To find out more about us, keep reading or feel free to get in touch.\n\nContact\n\nOur Educational Programs\n\nThe Best Day Care\n\nOutdoor Play\n\nArt\n\nImaginative Play\n\nPluto Day Care Mission\n\nUnique and Valuable\n\nWe believe that children thrive in an environment which values their own uniqueness, while providing support and opportunities to grow emotionally, socially and creatively. By building on each child’s strengths, interests and curiosities, we guide each child to explore the world around them as well as develop new abilities and form clos

In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split data
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)


print("Total number of documents: ",len(docs))

Total number of documents:  3


In [18]:
docs[0]

Document(metadata={'source': 'https://www.plutodaycare.com/'}, page_content='Welcome to Pluto Day Care\n\nYour Day Care\n\nAbout\n\nHow We Work\n\nAt Pluto Day Care, we are dedicated to creating a nurturing, stimulating and welcoming environment for all children. Our unique programs incorporate a variety of educational theories, while always keeping the child’s best interest in mind and allowing for hands-on, experiential learning. To find out more about us, keep reading or feel free to get in touch.\n\nContact\n\nOur Educational Programs\n\nThe Best Day Care\n\nOutdoor Play\n\nArt\n\nImaginative Play\n\nPluto Day Care Mission\n\nUnique and Valuable\n\nWe believe that children thrive in an environment which values their own uniqueness, while providing support and opportunities to grow emotionally, socially and creatively. By building on each child’s strengths, interests and curiosities, we guide each child to explore the world around them as well as develop new abilities and form close

## Setup the Embeddings

In [19]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import OpenAI
from dotenv import load_dotenv
load_dotenv(override=True)

vectorstore = Chroma.from_documents(documents=docs, embedding=OpenAIEmbeddings())

## Setup retriever

In [21]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.invoke("What kind of services they provide?")

In [22]:
len(retrieved_docs)

3

In [23]:
print(retrieved_docs[0].page_content)

About Pluto Day Care

A Place to Shine

With our progressive early childhood education philosophy, we understand that each child has unique needs and interests. We provide the environment and tools your child needs to thrive to the best of their abilities.

When we expose children to different environments and various learning experiences, we allow them to explore their self-awareness and gain a sense of the world around them. Once they leave Pluto Day Care, they’re fully prepared for the next stage of their young lives.

Get in Touch


## Setup Open AI LLM

In [24]:
llm = OpenAI(temperature=0.4, max_tokens=500)

In [25]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [26]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [27]:
response = rag_chain.invoke({"input": "Provide their address for all locations"})
print(response["answer"])

.

Pluto Day Care has three locations: 192 Activa Ave in Kitchener, 1418 Weber St. E. in Kitchener, and 435 King St. N. in Waterloo. Each location has a unique phone number and email address for contact.
